In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"theksg9","key":"5783530dd4c30706555b3c3f51310057"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d vipoooool/new-plant-diseases-dataset

100% 2.70G/2.70G [00:41<00:00, 29.8MB/s]
100% 2.70G/2.70G [00:41<00:00, 69.5MB/s]


In [ ]:
from zipfile import ZipFile
file_name="/content/new-plant-diseases-dataset.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale = 1./255,
	      rotation_range=90,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest'
        )

In [ ]:
!mv /content/dataset/dataset/train/Cherry* /content/dataset/cherry/train

In [ ]:
!mv /content/dataset/dataset/valid/Cherry* /content/dataset/cherry/test

In [ ]:
train_set=train_datagen.flow_from_directory(
    '/content/dataset/cherry/train',
    target_size=(64,64),
    batch_size=128,
    class_mode='categorical'
)

Found 3509 images belonging to 2 classes.


In [ ]:
test_datagen=ImageDataGenerator(rescale=1./255)

test_set=test_datagen.flow_from_directory(
    '/content/dataset/cherry/test',
    batch_size=32,
    target_size=(64,64),
    class_mode='categorical'
)

Found 877 images belonging to 2 classes.


In [ ]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
classifier=Sequential()
classifier.add(Convolution2D(16,(3,3),activation='relu',input_shape=(64,64,3)))
classifier.add(MaxPooling2D())
classifier.add(Convolution2D(32,(3,3),activation='relu'))
classifier.add(MaxPooling2D())
classifier.add(Flatten())
classifier.add(Dense(128,activation='relu'))
classifier.add(Dense(2,activation='softmax'))

In [ ]:
classifier.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
classifications=classifier.fit_generator(
    train_set,
    steps_per_epoch=3509/128,
    epochs=30,
    validation_data=test_set,
    validation_steps=877/32,
    verbose=2
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
27/27 - 41s - loss: 0.6927 - accuracy: 0.5987 - val_loss: 0.4941 - val_accuracy: 0.7651
Epoch 2/30
27/27 - 8s - loss: 0.4114 - accuracy: 0.8173 - val_loss: 0.3071 - val_accuracy: 0.8586
Epoch 3/30
27/27 - 8s - loss: 0.3190 - accuracy: 0.8661 - val_loss: 0.1947 - val_accuracy: 0.9339
Epoch 4/30
27/27 - 8s - loss: 0.2623 - accuracy: 0.9014 - val_loss: 0.1997 - val_accuracy: 0.9270
Epoch 5/30
27/27 - 8s - loss: 0.1838 - accuracy: 0.9302 - val_loss: 0.1041 - val_accuracy: 0.9726
Epoch 6/30
27/27 - 8s - loss: 0.1119 - accuracy: 0.9709 - val_loss: 0.1093 - val_accuracy: 0.9704
Epoch 7/30
27/27 - 8s - loss: 0.0925 - accuracy: 0.9724 - val_loss: 0.0883 - val_accuracy: 0.9829
Epoch 8/30
27/27 - 8s - loss: 0.0824 - accuracy: 0.9783 - val_loss: 0.1089 - val_accuracy: 0.9829
Epoch 9/30
27/27 - 8s - loss: 0.0820 - accuracy: 0.9732 - val_loss: 0.1854 - val_accuracy: 0.9475
Epoch 10/30
27/27 - 8s - loss: 0.0702 - accuracy: 0.9789 - val_loss: 0.0959 - val_accuracy: 0.9829
Epoch 11/30
27/27 

In [ ]:
import numpy as np
import cv2
from google.colab import files
from keras.preprocessing import image

uploaded=files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path='/content/' + fn
  img=cv2.imread(path, 1)
    
  img = cv2.resize(img, (64,64),interpolation = cv2.INTER_NEAREST)
  
  x=image.img_to_array(img)
  x=np.expand_dims(x, axis=0)
  images = np.vstack([x])
  images=images/255.0
  classes = classifier.predict(images, batch_size=10)
  print(classes[0])
  ar=np.array(classes[0]).tolist()
  max_value = max(ar)
  max_index = ar.index(max_value)
  
  if max_index==0 :
    print("PowderyMildew")
  else:
    print("Healthy")

Saving betula-lenta-le-ahaines-a.jpg to betula-lenta-le-ahaines-a.jpg
[1.5824632e-07 9.9999988e-01]
Healthy


In [ ]:
classifier.save('/content/drive/MyDrive/models/cherryCropNew.h5')